Python libraries used in this notebook.

In [ ]:
import pandas as pd
import requests


Generate map of census tracts for the 100th state congress district of Florida

In [ ]:
resp = requests.get('https://api.census.gov/data/2020/dec/pl?get=NAME&for=tract%20(or%20part):*&in=state:12%20state%20legislative%20district%20(lower%20chamber):100%20county%20(or%20part):086')
txt = resp.json()
map = pd.DataFrame(txt)

In [ ]:
new_header = map.iloc[0] 
map = map[1:] 
map.columns = new_header 
map.reset_index(inplace=True, drop=True)

In [ ]:
map['CensusTract'] = map['NAME']

In [ ]:
map['CensusTract'] = map['CensusTract'].str.replace("Census Tract ","") 
map['CensusTract'] = map['CensusTract'].str.split(',').str[0]
map['CensusTract'] = map['CensusTract'].str.replace("\(part\)","") 

Cleanup census household income.

In [ ]:
df = pd.read_excel(r'/home/jovyan/test_census_data/Input/ACSST5Y2020.S1901-2022-07-16T001726.xlsx')
df['CensusTract'] = df['CensusTract'].str.replace("Census Tract","") 
df['CensusTract'] = df['CensusTract'].str.replace(", Miami-Dade County, Florida","")
df['CensusTract'] = df['CensusTract'].str.strip()
df = df.set_index('CensusTract')


In [ ]:
display(df)

Back fill census tracts.

In [ ]:
df.index = pd.Series(df.index).fillna(method='ffill')

Remove all household types except for "household"

In [ ]:
df = df[df.HouseHoldType == 'Households']

Drop household type column.

In [ ]:
df.drop('HouseHoldType', inplace=True, axis=1)

In [ ]:
df = df.melt(var_name='IncomeGroup', value_name='Count', ignore_index=False)

display(df)

filter age and gender demo by florida 100th census tracts

In [ ]:
df = df[df.index.isin(map['CensusTract'])]

In [ ]:
display(df)

In [ ]:
df.to_excel('/home/jovyan/test_census_data/Output/output_income_miami_dade_100th.xlsx', sheet_name='Income')